<h1>Matrix functions</h1>
A set of function for matrices without numpy.

In [12]:
import math

def formatMatrix(M):
    return '[' + '\n '.join([' '.join(['{:7.3f}'.format(c) for c in row]) for row in M]) + ' ]'
    

def zeros (rows, cols):
    return [[0]*cols for i in range(rows)];


def identity (rows):
    M = zeros(rows, rows)
    for r in range(rows):
        M[r][r] = 1.0
    return M

def copyMatrix (M):
    return [[v for v in col] for col in M]

print ( formatMatrix( identity(3) ))

[  1.000   0.000   0.000
   0.000   1.000   0.000
   0.000   0.000   1.000 ]


<h2>Matrix transpose</h2>

In [2]:
def transpose(A):
    rowsA = len(A)
    colsA = len(A[0])
    
    return [[A[i][j] for i in range(rowsA)] for j in range(colsA)]

print (formatMatrix(transpose([[1,2,3],[4,5,6]])))

[  1.000   4.000
   2.000   5.000
   3.000   6.000 ]


<h2>Matrix multiplication</h2>

In [3]:
def scale(A, scale):
    return [[v*scale for v in col] for col in A]

def dot(A, B):
    rowsA = len(A)
    colsA = len(A[0])
    rowsB = len(B)
    colsB = len(B[0])
    
    if colsA != rowsB:
        raise Exception('Number of A columns must equal number of B rows.')
    
    C = zeros(rowsA, colsB)
    
    for i in range(rowsA):
        for j in range(colsB):
            C[i][j] = sum([A[i][k] * B[k][j] for k in range(colsA)])
            
    return C



A = [[1,2,3],[4,5,6]]
B = [[1,0],[1,1],[0,1]]
C = dot(A,B)
print ( formatMatrix( C ))
print ( formatMatrix( scale(A, 2.0) ))

[  3.000   5.000
   9.000  11.000 ]
[  2.000   4.000   6.000
   8.000  10.000  12.000 ]


<h2>Inverse a square matrix.</h2>

In [4]:
def inverse(A):
    rowsA = len(A)
    colsA = len(A[0])
    
    if rowsA != colsA:
        raise Exception('Matrix must be square')
    
    AM = copyMatrix(A)
    IM = identity(rowsA)
    
    for fd in range(rowsA):
        fdScaler = 1.0 / AM[fd][fd]
        for j in range(rowsA):
            AM[fd][j] *= fdScaler
            IM[fd][j] *= fdScaler
        for i in list(range(rowsA))[0:fd] + list(range(rowsA))[fd+1:]:
            crScaler = AM[i][fd]
            for j in range(rowsA):
                AM[i][j] = AM[i][j] - crScaler * AM[fd][j]
                IM[i][j] = IM[i][j] - crScaler * IM[fd][j]
    return IM
    
    
A = [[5,4,3,2,1],[4,3,2,1,5],[3,2,9,5,4],[2,1,5,4,3],[1,2,3,4,5]]
AI = inverse(A)

print ('A ::')
print ( formatMatrix( A ))
print ('Inverse ::')
print ( formatMatrix( AI ))
print ('A * inverse :: ')
print ( formatMatrix(dot( A, AI )))

A ::
[  5.000   4.000   3.000   2.000   1.000
   4.000   3.000   2.000   1.000   5.000
   3.000   2.000   9.000   5.000   4.000
   2.000   1.000   5.000   4.000   3.000
   1.000   2.000   3.000   4.000   5.000 ]
Inverse ::
[  0.022   0.200  -0.333   0.733  -0.378
   0.244  -0.200   0.333  -0.933   0.444
  -0.056  -0.000   0.333  -0.333  -0.056
   0.122  -0.200  -0.333   0.533   0.122
  -0.167   0.200   0.000  -0.000   0.033 ]
A * inverse :: 
[  1.000   0.000  -0.000  -0.000  -0.000
   0.000   1.000   0.000  -0.000   0.000
   0.000   0.000   1.000  -0.000  -0.000
   0.000   0.000   0.000   1.000  -0.000
   0.000   0.000  -0.000  -0.000   1.000 ]


<h2>Solve the Ax=b linear equations system.</h2> <br>
By taking the inverse of A and multiplying b.

In [5]:
def solve(A, b):
    Inv = inverse(A)
    return dot(Inv, b)


# 4x + 3y = -13
# -10x -2y = 5

A = [[4, 3], [-10, -2]]
b = [[-13], [5]]

x = solve(A, b)
print ( formatMatrix(x) )

A = [[4, 3], [-10, -2]]
b = [[-2], [1]]

x = solve(A, b)
print ( formatMatrix(x) )

A = [[4, 3], [-10, -2]]
b = [[-13,-2], [5,1]]

x = solve(A, b)
print ( formatMatrix(x) )

[  0.500
  -5.000 ]
[  0.045
  -0.727 ]
[  0.500   0.045
  -5.000  -0.727 ]


# Orthogonalization.


In [27]:
def orthogonize (A):
    
    def _dot(a,b):
        return sum([ia * ib for ia,ib in zip(a,b)])
    
    def _proj(a,b):
        coef = _dot(b,a) / _dot(a,a)
        return [ia * coef for ia in a]
    
    rowsA = len(A)
    colsA = len(A[0])
    B = []
    for i in range(rowsA):
        temp_vec = A[i]
        for inB in B :
            proj_vec = _proj(inB, A[i])
            temp_vec = [x-y for x,y in zip(temp_vec, proj_vec)]
        coef = 1.0/math.sqrt(_dot(temp_vec,temp_vec))
        temp_vec = [it*coef for it in temp_vec]
        B.append(temp_vec)
    return B


#tests
A = [
    [1,0,0],
    [0,1,0],
    [0,0,1]
]
print ( formatMatrix(orthogonize(A)))

A = [
    [1.1,0,0],
    [0,1.2,0],
    [0,0,1.1]
]
print ( formatMatrix(orthogonize(A)))

A = [
    [1,0.1,0],
    [0,1,0],
    [0,0,1]
]
print ( formatMatrix(orthogonize(A)))


[  1.000   0.000   0.000
   0.000   1.000   0.000
   0.000   0.000   1.000 ]
[  1.000   0.000   0.000
   0.000   1.000   0.000
   0.000   0.000   1.000 ]
[  0.995   0.100   0.000
  -0.100   0.995   0.000
   0.000   0.000   1.000 ]
